## Testing

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import numpy as np
import pandas as pd
import cv2
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D, Conv2D, UpSampling2D, MaxPool2D, AveragePooling2D, Flatten, Dropout, BatchNormalization, MaxPooling2D, Activation, LSTM 
from tensorflow.keras.layers import SeparableConv2D, Reshape, Input, Lambda, Concatenate
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import RMSprop, SGD, Adam, Nadam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import L2
from adabelief_tf import AdaBeliefOptimizer

In [ ]:
def load_data(data_path):
    data = pd.read_csv(data_path)
    pixels = data['pixels'].tolist()
    width, height = 48, 48
    faces = []
    for pixel_sequence in pixels:
        face = [int(pixel) for pixel in pixel_sequence.split(' ')]
        face = np.asarray(face).reshape(width, height)
        face = cv2.resize(face.astype('uint8'),image_size)
        faces.append(face.astype('float32'))
    faces = np.asarray(faces)
    faces = np.expand_dims(faces, -1)

    emotions = pd.get_dummies(data['emotion']).values
    return faces, emotions

In [ ]:
# Model reconstruction from JSON file
with open('Brendon.json', 'r') as json_file:
    json_savedModel= json_file.read()

test_model = tf.keras.models.model_from_json(json_savedModel)
test_model.summary()

optimizer = Nadam(learning_rate=1e-3, epsilon=1e-7)

test_model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['acc'])

# Load weights into the new model
test_model.load_weights('./Brendon.h5')

In [ ]:
test_path = 'test.csv'
image_size= (48,48)

faces_test, emotions_test = load_data(test_path)

In [ ]:
test_loss, test_acc = test_model.evaluate(faces_test/255.0, emotions_test) 
print('Test accuracy:', test_acc)